In [ ]:
from collections import defaultdict
from copy import copy
from functools import partial
from operator import itemgetter

import adaptive
import holoviews as hv
import numpy as np

import funcs
from common import loss
import adaptive_tools

hv.notebook_extension('bokeh')

%opts Image {+axiswise} [colorbar=True aspect='square'] (cmap='viridis')

def f(x): pass

def select(learners, args):
    return next(l for l in learners
                if tuple(l.cdims.values()) == tuple(args))

def plot_learners(learners, *args):
    learner = select(learners, args)
    return learner.plot(tri_alpha=0).redim(x='V_bias', y='B_x', z='G')

def plot_learners_angle(learners, *args):
    learner = select(learners, args)
    return learner.plot(tri_alpha=0).redim(x='V_bias', y='angle', z='G')

def get_dm(learners, plot_function):
    d = defaultdict(list)
    for learner in learners:
        for k, v in learner.cdims.items():
            d[k].append(v)
    dm = hv.DynamicMap(partial(plot_function, learners), kdims=list(d.keys()))
    return dm.redim.values(**d)

# Chemical potential sweep

### _Note: in the low chemical potential regime the lines are so thin that they are not found._ 

In [ ]:
syst_pars = dict(a=10, angle=0, onsite_disorder=False,
                 L=2000, coverage_angle=180, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True)

params = dict(Delta=85.875, c_tunnel=3/4, V='lambda x, y, z: 0', mu_sc=100,
              V_barrier=40,
             **funcs.constants.__dict__)

vals = funcs.named_product(g=[0, 50],
                           alpha=[0, 20],
                           orbital=[False, True],
                           direction=list('xyz'),
                           mu=list(range(10, 21)))

learners = []
for val in vals:
    learner = adaptive_tools.Learner2D(f, [(-1, 1), (0, 2)])
    learner.cdims = val
    learners.append(learner)

learner = adaptive_tools.BalancingLearner(learners)
folder = 'data/mu-sweep2'
learner.load(folder, 'data_learner_{}.pickle')

In [ ]:
hv.notebook_extension('matplotlib')

In [ ]:
%%opts Image {+framewise}
%%output size=200
dm = get_dm(learners, plot_learners)
dm

In [ ]:
sum([l.npoints for l in learners]) / len(learners), len(learners)

# Electrical field sweep at fixed $\mu$

In [ ]:
gaps = {-10: {0: 201.375, 90: 125.875},
        -8: {0: 178.375, 90: 116.625},
        -6: {0: 152.875, 90: 110.125},
        -4: {0: 129.625, 90: 103.125},
        -2: {0: 107.125, 90: 97.375},
         0: {0: 85.875, 90: 94.875},
         2: {0: 65.625, 90: 93.625},
         4: {0: 46.875, 90: 94.625},
         6: {0: 30.125, 90: 97.375},
       }

params = dict(c_tunnel=3/4, V_barrier=40, mu_sc=100, mu=15,
             **funcs.constants.__dict__)

syst_pars = dict(a=10, onsite_disorder=False, 
                 L=2000, coverage_angle=180, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True)

vals = funcs.named_product(g=[0, 50],
                           alpha=[0, 20],
                           orbital=[False, True],
                           direction=list('xyz'),
                           gradient=list(gaps.keys()),
                           angle=[0, 90])

learners = []
for val in vals:
    f = lambda x: 0
    learner = adaptive_tools.Learner2D(f, [(-1, 1), (0, 2)])
    learner.cdims = val
    learners.append(learner)

learner = adaptive_tools.BalancingLearner(learners)
folder = 'data/gradient-sweep-angle-0-90'
learner.load(folder, 'data_learner_{}.pickle')

In [ ]:
%%opts Image {+framewise} [colorbar=True]
%%output size=200

dm = get_dm(learners, plot_learners_angle)
dm

In [ ]:
# hm.layout('angle')  # can't use DynamicMap because of https://github.com/ioam/holoviews/issues/2251

# Rotate field from Z to Y

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4f/3D_Spherical.svg/208px-3D_Spherical.svg.png)

In [ ]:
def conductance_rotation(x, val, syst_pars, params):
    import funcs, common

    for k in ['angle']:
        syst_pars[k] = val[k]
    
    params['mu_lead'] = params['mu']
    val['V_bias'], theta = x
    val['B_x'], val['B_y'], val['B_z'] = common.spherical_coords(
        params['B'], theta, 90)

    params = funcs.parse_params(dict(**params, **val))

    syst = funcs.make_3d_wire(**syst_pars)
    return funcs.conductance(syst, params, E=val['V_bias'])

gaps = {-10: {0: 193.625, 45: 113.125},
        -8: {0: 165.875, 45: 96.625},
        -6: {0: 147.375, 45: 79.125},
        -4: {0: 119.875, 45: 63.125},
        -2: {0: 98.625, 45: 46.625},
        0: {0: 78.125, 45: 32.125},
        2: {0: 59.875, 45: 20.875},
        4: {0: 39.375, 45: 13.125}}

params = dict(c_tunnel=3/4, V_barrier=40, mu_sc=100, mu=15, B=0.25,
             **funcs.constants.__dict__)

syst_pars = dict(a=10, onsite_disorder=False, 
                 L=2000, coverage_angle=135, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True)

vals = funcs.named_product(g=[0, 50],
                           alpha=[0, 20],
                           orbital=[False, True],
                           gradient=list(gaps.keys()),
                           angle=[0, 45])

learners = []
for val in vals:
    params['V'] = f'lambda x, y, z: {val["gradient"]} * z / {syst_pars["r1"]}'
    params['Delta'] = gaps[val['gradient']][val['angle']]
    f = partial(conductance_rotation, val=val, params=copy(params), syst_pars=copy(syst_pars))
    learner = adaptive_tools.Learner2D(f, [(-1, 1), (-90, 135)])
    learner.cdims = val
    learner.stack_size = 40
    learners.append(learner)

learner = adaptive_tools.BalancingLearner(learners)
folder = 'data/gradient-sweep-rotation-0-45'
learner.load(folder, 'data_learner_{}.pickle')

In [ ]:
%%opts Image {+framewise} [colorbar=True] (cmap='gist_heat')
%%output size=200

get_dm(learners, plot_learners_angle)

# Rotation with moving SO and potential

In [ ]:
def conductance_rotation(x, val, syst_pars, params):
    import funcs, common

    for k in ['angle']:
        syst_pars[k] = val[k]
    
    params['mu_lead'] = params['mu']
    val['V_bias'], theta = x
    val['B_x'], val['B_y'], val['B_z'] = common.spherical_coords(
        params['B'], theta, 90)

    params = funcs.parse_params(dict(**params, **val))

    syst = funcs.make_3d_wire(**syst_pars)
    return funcs.conductance(syst, params, E=val['V_bias'])

gaps = {-10: {0: 193.625, 45: 113.125},
        -8: {0: 165.875, 45: 96.625},
        -6: {0: 147.375, 45: 79.125},
        -4: {0: 119.875, 45: 63.125},
        -2: {0: 98.625, 45: 46.625},
        0: {0: 78.125, 45: 32.125},
        2: {0: 59.875, 45: 20.875},
        4: {0: 39.375, 45: 13.125}}

params = dict(c_tunnel=3/4, V_barrier=40, mu_sc=100, mu=15, B=0.25,
              sin='np.sin', cos='np.cos', **funcs.constants.__dict__)

syst_pars = dict(a=10, onsite_disorder=False, 
                 L=2000, coverage_angle=135, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True,
                 rotate_spin_orbit=True)

vals = funcs.named_product(g=[0, 50],
                           alpha=[0, 20],
                           orbital=[True, False],
                           gradient=list(gaps.keys()),
                           angle=[0, 45],
                           theta_SO=np.deg2rad([90, 45]),
                           rotate_V=[True, False])

learners = []
for val in vals:
    if val['rotate_V']:
        coord = f'np.sin({val["theta_SO"]}) * z + np.cos({val["theta_SO"]}) * y'
    else:
        coord = 'z'
    params['V'] = f'lambda x, y, z: {val["gradient"]} * ({coord}) / {syst_pars["r1"]}'

    params['Delta'] = gaps[val['gradient']][val['angle']]
    f = partial(conductance_rotation, val=val, params=copy(params), syst_pars=copy(syst_pars))
    learner = adaptive_tools.Learner2D(f, [(-1, 1), (-90, 135)])
    learner.cdims = val
    learner.stack_size = 40
    learners.append(learner)

learner = adaptive_tools.BalancingLearner(learners)
folder = 'data/gradient-sweep-rotation-0-45-move-SO-and-potential'
learner.load(folder, 'data_learner_{}.pickle')

In [ ]:
%%opts Image {+framewise} [colorbar=True] (cmap='gist_heat')
%%output size=200

get_dm(learners, plot_learners_angle) * hv.HLine(0) * hv.HLine(45)

# Change angle of spin-orbit

In [ ]:
def conductance_rotation(x, val, syst_pars, params):
    import funcs, common

    for k in ['angle']:
        syst_pars[k] = val[k]
    
    params['mu_lead'] = params['mu']
    val['V_bias'], theta = x
    val['B_x'], val['B_y'], val['B_z'] = common.spherical_coords(
        params['B'], theta, 90)

    params = funcs.parse_params(dict(**params, **val))

    syst = funcs.make_3d_wire(**syst_pars)
    return funcs.conductance(syst, params, E=val['V_bias'])

gaps = {-10: {0: 193.625, 45: 113.125},
        -8: {0: 165.875, 45: 96.625},
        -6: {0: 147.375, 45: 79.125},
        -4: {0: 119.875, 45: 63.125},
        -2: {0: 98.625, 45: 46.625},
        0: {0: 78.125, 45: 32.125},
        2: {0: 59.875, 45: 20.875},
        4: {0: 39.375, 45: 13.125}}

params = dict(c_tunnel=3/4, V_barrier=40, mu_sc=100, mu=15, B=0.25,
              sin='lambda x: np.sin(np.deg2rad(x))', cos='lambda x: np.cos(np.deg2rad(x))',
              **funcs.constants.__dict__)

syst_pars = dict(a=10, onsite_disorder=False, 
                 L=2000, coverage_angle=135, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True,
                 rotate_spin_orbit=True)

vals = funcs.named_product(g=[0, 50],
                           alpha=[0, 20],
                           orbital=[True, False],
                           gradient=list(gaps.keys()),
                           angle=[0, 45],
                           theta_SO=np.linspace(0, 90, 7),
                           theta_V=[90])

learners = []
for val in vals:
    gradient = val["gradient"]
    r = syst_pars["r1"]
    theta_V = np.deg2rad(val["theta_V"])
    coord = f'np.sin({theta_V}) * z + np.cos({theta_V}) * y'
    params['V'] = f'lambda x, y, z: {gradient} * ({coord}) / {r}'
    params['Delta'] = gaps[gradient][val['angle']]

    f = partial(conductance_rotation, val=val, params=copy(params), syst_pars=copy(syst_pars))
    learner = adaptive_tools.Learner2D(f, [(-1, 1), (-90, 135)], loss)
    learner.cdims = val
    learners.append(learner)

learner = adaptive_tools.BalancingLearner(learners)
folder = 'data/gradient-sweep-rotation-0-90-move-SO-slowly'
learner.load(folder, 'data_learner_{}.pickle')

In [ ]:
%%opts Image {+framewise} [colorbar=True]
%%output size=200

get_dm(learners, plot_learners_angle)

# Change alpha

In [ ]:
gaps = {-10: {0: 193.625, 45: 113.125},
        -8: {0: 165.875, 45: 96.625},
        -6: {0: 147.375, 45: 79.125},
        -4: {0: 119.875, 45: 63.125},
        -2: {0: 98.625, 45: 46.625},
        0: {0: 78.125, 45: 32.125},
        2: {0: 59.875, 45: 20.875},
        4: {0: 39.375, 45: 13.125}}

params = dict(c_tunnel=3/4, V_barrier=40, mu_sc=100, mu=15,
             **funcs.constants.__dict__)

syst_pars = dict(a=10, onsite_disorder=False, 
                 L=2000, coverage_angle=135, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True)

vals = funcs.named_product(g=[0, 50],
                           alpha=[0, 20, 50, 100, 250],
                           orbital=[False, True],
                           direction=list('xy'),
                           gradient=list(gaps.keys()),
                           angle=[0])

learners = []
for val in vals:
    params['V'] = f'lambda x, y, z: {val["gradient"]} * z / {syst_pars["r1"]}'
    params['Delta'] = gaps[val['gradient']][val['angle']]
    
    f = lambda *_: 0 # partial(conductance, val=val, params=copy(params), syst_pars=copy(syst_pars))
    learner = adaptive_tools.Learner2D(f, [(-1, 1), (0, 2)], loss)
    learner.cdims = val
    learners.append(learner)

learner = adaptive_tools.BalancingLearner(learners)
folder = 'data/gradient-sweep-alpha'
learner.load(folder, 'data_learner_{}.pickle')

In [ ]:
%%opts Image {+framewise} [colorbar=True] (cmap='gist_heat')
%%output size=200

dm = get_dm(learners, plot_learners)
dm

In [ ]:
gaps = {-10: {0: 193.625, 45: 113.125},
        -8: {0: 165.875, 45: 96.625},
        -6: {0: 147.375, 45: 79.125},
        -4: {0: 119.875, 45: 63.125},
        -2: {0: 98.625, 45: 46.625},
        0: {0: 78.125, 45: 32.125},
        2: {0: 59.875, 45: 20.875},
        4: {0: 39.375, 45: 13.125}}

params = dict(c_tunnel=3/4, V_barrier=40, mu_sc=100,
             **funcs.constants.__dict__)

syst_pars = dict(a=10, onsite_disorder=False, 
                 L=2000, coverage_angle=135, r1=35, r2=70, shape='circle',
                 with_leads=True, with_shell=True, A_correction=True)

vals = funcs.named_product(g=[0, 50],
                           alpha=[0, 20, 50, 100, 250],
                           orbital=[False, True],
                           direction=list('xy'),
                           gradient=list(gaps.keys()),
                           mu=[10, 12, 15],
                           angle=[0])

learners = []
for val in vals:
    params['V'] = f'lambda x, y, z: {val["gradient"]} * z / {syst_pars["r1"]}'
    params['Delta'] = gaps[val['gradient']][val['angle']]
    f = lambda x: 0
#     f = partial(conductance, val=val, params=copy(params), syst_pars=copy(syst_pars))
    learner = Learner2D(f, [(-1, 1), (0, 2)], loss)
    learner.cdims = val
    learners.append(learner)

learner = BalancingLearner(learners)
folder = 'data/gradient-sweep-alpha3'
learner.load(folder, 'data_learner_{}.pickle')